<a href="https://colab.research.google.com/github/Minh-Khoa-Pham/Housing-Price/blob/main/Big_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import patsy as pt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:

#Collect and clean main data about housing price in Ireland
data = pd.read_csv('https://raw.githubusercontent.com/Minh-Khoa-Pham/Housing-Price/main/HousingPrice.csv')
data = data.rename(columns={'QIEN628BIS':'USD'})
data['USD'] = data['USD']*1000
#Only keep data for month October and remove year 2021 & 2022
data = data[data.DATE.str.match('^\d{4}-10-\d{2}')]
data['Year'] = pd.DatetimeIndex(data['DATE']).year
data.drop('DATE',axis=1,inplace=True)
data.drop(207,axis=0,inplace=True)
#reset index
data = data.reset_index()
data.drop('index',axis=1,inplace=True)
#change position of columns
data = data[['Year','USD']]
#drop years 1970 to 1975
data.drop(data.index[0:5],axis=0,inplace=True)
data = data.reset_index()
data.drop('index',axis=1,inplace=True)

#Collect data for Irish GDP per capita
gdp = pd.read_csv('https://raw.githubusercontent.com/Minh-Khoa-Pham/Housing-Price/main/GDP.csv')
gdp = gdp.rename(columns={'label':'Year','GDP Per Capita':'GDPpercap'})
#Reverse rows using loc() Function
gdp = gdp.loc[::-1]
#Clean up data from the reversed dataframe
gdp = gdp.reset_index()
gdp.drop('index',axis=1,inplace=True)
gdp.drop(data.index[0:15],axis=0,inplace=True)
gdp = gdp.reset_index()
gdp.drop('index',axis=1,inplace=True)

In [ ]:
#Collect data for Mortage Interest Rates
interest = pd.read_csv('https://raw.githubusercontent.com/Minh-Khoa-Pham/Housing-Price/main/mortgage%20rates.csv')
interest['Highest Mortgage Interest Rate'] = interest['Highest Mortgage Interest Rate'].str.rstrip('%').astype('float')
interest = interest.rename(columns={'Highest Mortgage Interest Rate':'Mortgage Interest Rate in %'})

# Create figure with secondary y-axis
for template in ['simple_white']:
  fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
  fig2.add_trace(
      go.Scatter(x=interest['Year'], y=interest['Mortgage Interest Rate in %'], name="Mortgage Interest Rate (%)"),
      secondary_y=True,)

  fig2.add_trace(
      go.Scatter(x=data['Year'], y=data['USD'], name="Residential property prices (RPPI)"),
      secondary_y=False,)

# Add figure title and template
  fig2.update_layout(
      title_text="<b>Housing Prices and Mortgage Rates</b>")
  fig2.update_layout(template=template)

# Set x-axis title
  fig2.update_xaxes(title_text="Years")

# Set y-axes titles
  fig2.update_yaxes(title_text="Residential property prices (RPPI)", secondary_y=False)
  fig2.update_yaxes(title_text="Mortgage Interest Rate (%)", secondary_y=True)

  fig2.show()

In [ ]:
# Collect data about inflation in Ireland
inflation = pd.read_csv('https://raw.githubusercontent.com/Minh-Khoa-Pham/Housing-Price/main/inflation_data.csv')
inflation = inflation.rename(columns={'year':'Year'})

# Create figure with secondary y-axis
for template in ['simple_white']:
  fig3 = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
  fig3.add_trace(
      go.Scatter(x=interest['Year'], y=inflation['amount'], name="Euro Inflation Amount (€)"),
      secondary_y=True,)

  fig3.add_trace(
      go.Scatter(x=data['Year'], y=data['USD'], name="Residential property prices (RPPI)"),
      secondary_y=False,)

# Add figure title and template
  fig3.update_layout(
      title_text="<b>Housing Prices and Inflation Amount</b>")
  fig3.update_layout(template=template)

# Set x-axis title
  fig3.update_xaxes(title_text="Years")

# Set y-axes titles
  fig3.update_yaxes(title_text="Residential property prices (RPPI)", secondary_y=False)
  fig3.update_yaxes(title_text="Euro Inflation Amount (€)", secondary_y=True)

  fig3.show()

In [ ]:
#Plot dual axes for housing prices and GDP per capita growth
for template in ['simple_white']:
    fig4 = make_subplots(specs=[[{"secondary_y": True}]])
    fig4.add_trace(
        go.Scatter(x=data['Year'], y=gdp['GDPpercap'], name="GDP per capita (USD)"),
        secondary_y=True)
    fig4.add_trace(
        go.Scatter(x=data['Year'], y=data['USD'], name="Residential property prices (RPPI)"),
        secondary_y=False,)
    fig4.update_layout(
        title_text="Comparing Housing prices and GDP per capita of Ireland")
    fig4.update_xaxes(title_text="Years")
    fig4.update_yaxes(title_text="Residential property prices (RPPI)", secondary_y=False)
    fig4.update_yaxes(title_text="GDP per capita (USD)", secondary_y=True)
    fig4.update_layout(template=template)
fig4.show()

In [ ]:
#Collect data about Population in Ireland
population = pd.read_csv('https://raw.githubusercontent.com/Minh-Khoa-Pham/Housing-Price/main/ireland%20population.csv')
population.head()

#Only keep data for year and remove months and days
population['date'] = pd.DatetimeIndex(population['date']).year
population.dtypes
population.head()


# Create figure with secondary y-axis
for template in ['simple_white']:
  fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
  fig.add_trace(
      go.Scatter(x=population['date'], y=population[' Population'], name="Population Growth"),
      secondary_y=True,
)

  fig.add_trace(
      go.Scatter(x=data['Year'], y=data['USD'], name="Residential property prices (RPPI)"),
      secondary_y=False,
)

# Add figure title
  fig.update_layout(
      title_text="<b>Housing Prices and Population</b>"
)
  fig.update_layout(template=template)

# Set x-axis title
  fig.update_xaxes(title_text="Years")

# Set y-axes titles
  fig.update_yaxes(title_text="Residential property prices (RPPI)", secondary_y=False)
  fig.update_yaxes(title_text="Population in Ireland", secondary_y=True)

fig.show()


In [ ]:
#concatenate the dataframes
data['GDPpercap'] = gdp['GDPpercap']
data['mortgage_interest_rate'] = interest['Mortgage Interest Rate in %']
data['population'] = population[' Population']
data['inflation'] = inflation['amount']

In [ ]:
data.head()

,Year,USD,GDPpercap,mortgage_interest_rate,population,inflation
0,1975,6297.0,2973.866039,12.50,3157919,100.00
1,1976,7764.7,2920.040691,13.95,3205753,117.99
2,1977,9082.1,3427.610455,13.96,3254662,133.88
3,1978,11343.1,4400.676295,14.15,3305571,144.20
4,1979,13631.9,5430.813380,14.15,3351898,163.38


In [ ]:
# Linear regression analysis to predict housing prices based on mortgage rates, GDP, and population
reg = smf.ols('USD ~ GDPpercap + mortgage_interest_rate + population + inflation', data=data).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                    USD   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     128.6
Date:                Wed, 10 Aug 2022   Prob (F-statistic):           1.25e-22
Time:                        04:19:47   Log-Likelihood:                -498.12
No. Observations:                  46   AIC:                             1006.
Df Residuals:                      41   BIC:                             1015.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               2.91